# [Beta Distribution](https://en.wikipedia.org/wiki/Beta_distribution)


Here is a light introduction to Beta Distribution. Beta Distribution is has two parameter $a,b > 0$. For $0 \leqslant x \leqslant 1$ PDF of beta is as follows

$$f_{a,b}(x)=\frac {1}{B(a,b)}x^{a-1}(1-x){b-1},$$

where $B(a,b)$ is the normalizer and equals to $\frac {\Gamma (a)\Gamma(b)}{\Gamma (a+b)}$. To see why this quantity is the normalizer look for example at [this](https://brilliant.org/wiki/beta-function/).

But First of all lets play a bit with parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets.widgets import IntSlider,Checkbox,fixed,FloatSlider
from scipy.stats import beta,binom
%matplotlib inline

In [5]:
@interact(a=FloatSlider(min=0.1,max=5,value=1,step=0.1),b=FloatSlider(min=0.1,max=5,value=1,step=0.1))
def mybeta(a,b):
    RV=beta(a,b)
    x=np.linspace(0,1,100)
    
    plt.plot(x,RV.pdf(x))
    print('Mean : ', RV.expect(), '   |    Std : ', RV.std())

interactive(children=(FloatSlider(value=1.0, description='a', max=5.0, min=0.1), FloatSlider(value=1.0, description='b', max=5.0, min=0.1), Output()), _dom_classes=('widget-interact',))

### [Polya Urn](https://en.wikipedia.org/wiki/P%C3%B3lya_urn_model)

Probably it's the most intutive way to think about Beta. Consider an urn containing $a$ white balls and $b$ black balls, $a,b$ are not neccessarily integer! You may think of it as mass instead of number. At each step, draw a ball uniformly at random and replace it together with another ball of the same color. Percentage of white balls, i.e. $\frac {a}{a+b}$, converges to $\beta(a,b)$.

Here is a simulation.

In [38]:
@interact(a=FloatSlider(min=0.1,max=5,value=1,step=0.1),b=FloatSlider(min=0.1,max=5,value=1,step=0.1))
def mybeta(a,b):
    N=8
    it=11
    r_list=[]
    for t in range(2**it):
        aa=a
        bb=b
        for i in range(2**N):
            if np.random.rand()<aa/(aa+bb):
                aa+=1
            else:
                bb+=1
        r_list.append(aa/(aa+bb))
    
    RV=beta(a,b)
    x=np.linspace(0,1,100)
    plt.plot(x,RV.pdf(x))
    plt.hist(r_list,bins=np.linspace(0,1,10),density=True)

interactive(children=(FloatSlider(value=1.0, description='a', max=5.0, min=0.1), FloatSlider(value=1.0, description='b', max=5.0, min=0.1), Output()), _dom_classes=('widget-interact',))

### [Order statistics](https://en.wikipedia.org/wiki/Order_statistic#The_order_statistics_of_the_uniform_distribution)

Sample $n$ numbers in $[0,1]$ uniformly at random. What is the distribution of the $k-$th smallest one? The answer is $\beta(k,n-k+1)$. I couldn't find any direct relation between Polya urn and Order statistics, but to see a proof you may look at [this](https://stats.stackexchange.com/questions/4659/relationship-between-binomial-and-beta-distributions/).

Here is the simulation.

In [43]:
@interact(n=IntSlider(min=1,max=20),k=IntSlider(min=1,max=20))
def mybeta(n,k):
    it=18
    A=np.sort(np.random.rand(2**it,n))
    l=A[:,k-1]
    RV=beta(k,n-k+1)
    x=np.linspace(0,1,100)
    plt.plot(x,RV.pdf(x))
    plt.hist(l,bins=np.linspace(0,1,100),density=True)

interactive(children=(IntSlider(value=1, description='n', max=20, min=1), IntSlider(value=1, description='k', max=20, min=1), Output()), _dom_classes=('widget-interact',))

### Relation to [Binomial Distribution](https://en.wikipedia.org/wiki/Binomial_distribution)

It is often said that Beta distribution is a distribution on distributions! But what is means?

It essentially means that you may fix $n,k$ and think of $\mathbb P[Bin(n,p)\geqslant k]$ as a function of $p$. What the calculation below says is that the value of $\mathbb P[Bin(n,p)\geqslant k]$ increases from $0$ to $1$ when you tune $p$ from $0$ to $1$. The increasing rate at each $p$ is exactly $\beta(k,n-k+1)$ at that $p$.

------------------

Let $Bin(n,p)$ denote a Binomial random variable with $n$ samples and the probability of success $p$. Using basic algebra we have

$$\frac d{dp}\mathbb P[Bin(n,p)=i]=n\Big(\mathbb P[Bin(n-1,p)=i-1]-\mathbb P[Bin(n-1,p)=i]\Big).$$

*It has also some nice combinatorial proof, think of it as an exercise!*

So, we have:

$$\frac d{dp}\mathbb P[Bin(n,p)\geqslant k]=\frac d{dp}\sum_{i=k}^{n}\mathbb P[Bin(n,p)=i]=n\Big(\sum_{i=k}^{n}\mathbb P[Bin(n-1,p)=i-1]-\mathbb P[Bin(n-1,p)=i]\Big)$$
which is a telescoping series and can be simplified as

$$\frac d{dp}\mathbb P[Bin(n,p)\geqslant k]=n\mathbb P[Bin(n-1,p)=k-1]=\frac{n!}{(k-1)!(n-k)!}p^{k-1}(1-p)^{n-k}=\beta(k,n-k+1).$$

As desired.

----------------

Here are two implementaion. The first one is easier to understand. The blue dotted plot is just $y=P[Bin(n,p)>=k+1]$ and the orange dotted plot is its derivative. You should check the checkbox to see it. The blue curve is $\beta (k+1,n-k)$ and as you can see in coincide with the orange plot.

In [48]:
@interact(n=IntSlider(min=1,max=50,value=20),k=IntSlider(min=0,max=50,value=10),derivative=Checkbox())
def myder(n,k,derivative):
    ss=0.01
    xp=np.arange(0,1,ss)
    y=[]
    Bt=beta(k+1,n-k)
    for p in xp:
        Bn=binom(n,p)
        y.append(1-Bn.cdf(k))
    dy=np.diff(y)
    plt.figure(figsize=(12,6))
    plt.scatter(xp,y)
    plt.legend(['y=P[Bin({},p)>{}]'.format(n,k)])
    if derivative:
        plt.scatter(ss/2+xp[:-1],dy/ss)
        plt.plot(xp,Bt.pdf(xp))
        plt.legend(['beta({},{})'.format(k+1,n-k),'y=P[Bin({},p)>={}]'.format(n,k+1),'dy/dp'])

interactive(children=(IntSlider(value=20, description='n', max=50, min=1), IntSlider(value=10, description='k', max=50), Checkbox(value=False, description='derivative'), Output()), _dom_classes=('widget-interact',))

The second implementaion is also interesting since you may also tune $p$. Here the plot on the left is CDF of $Bin(n,p)$ and the cyan dot, shows its derivative according to $p$ at the point $x=k$. (Note that $p$ is not the $x-$ axis of the plot, so it's not exactly the slope of the tangent line there.)

The right hand side graph is $\beta (k,n-k+1)$, and you can see another cyan dot whose $y$ coordinate is sync with the previous one but its $x$ coordinate is $p$. You may see that this point is moving on beta distribution.

In [53]:
ss=0.01
@interact(p=FloatSlider(min=ss,max=1,step=ss,value=0.5),n=IntSlider(min=1,max=50,value=20),k=IntSlider(min=1,max=50,value=10))
def myder2(p,n,k):
    Bt=beta(k,n-k+1)
    Bn=binom(n,p)
    Bn2=binom(n,p-ss)
    xbn=np.arange(0,n+1)
    plt.figure(figsize=(15,4))
    
    plt.subplot(1,2,1)
    plt.plot(xbn,Bn.cdf(xbn))
    der=(-Bn.cdf(k-1)+Bn2.cdf(k-1))/ss
    plt.scatter(x=k-1,y=der,lw=5,c='c')
    plt.vlines(k-1,0,Bn.cdf(k-1),'gray','--')
    plt.hlines(der,k-1,n,'gray','--')
    plt.ylim([-0.1,5.1])
    
    plt.subplot(1,2,2)
    xp=np.arange(0,1,ss)
    plt.plot(xp,Bt.pdf(xp))
    plt.scatter(x=p,y=der,lw=5,c='c')
    plt.hlines(der,0,p,'gray','--')
    plt.ylim([-0.1,5.1])

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0, min=0.01, step=0.01), IntSlider(value=20, description='n', max=50, min=1), IntSlider(value=10, description='k', max=50, min=1), Output()), _dom_classes=('widget-interact',))